In [1]:
import pandas as pd
import os
import folium
import subprocess
from sklearn import preprocessing

print(folium.__version__)

0.5.0


In [4]:
df = pd.read_csv('data/campaign_contributions_for_map.tsv', sep='\t')
df.sort_values('Amount', ascending=False).head()

FileNotFoundError: File b'data/campaign_contributions_for_map.tsv' does not exist

In [ ]:
df.info()

In [3]:
valid_heatmap_df = df[["lat", "lon", "Amount"]]
valid_heatmap_df.head()

NameError: name 'df' is not defined

## Heatmap

In [5]:
# convert pandas.DataFrame to numpy.array
data = valid_heatmap_df[["lat", "lon", "Amount"]].values

In [6]:
# normalize weight values to [0, 1]
min_max_scaler = preprocessing.MinMaxScaler()
data[:, [2]] = min_max_scaler.fit_transform(data[:, [2]])

In [7]:
# map params
location = [41.0, -75.5]
zoom_start = 8
# heatmap params
# layer name
name = "layer01"
max_val = data[:, 2].max()
min_opacity = 0.9
blur = 20
radius = 12
help(folium.Map)

Help on class Map in module folium.folium:

class Map(folium.map.LegacyMap)
 |  Create a Map with Folium and Leaflet.js
 |  
 |  Generate a base map of given width and height with either default
 |  tilesets or a custom tileset URL. The following tilesets are built-in
 |  to Folium. Pass any of the following to the "tiles" keyword:
 |  
 |      - "OpenStreetMap"
 |      - "Mapbox Bright" (Limited levels of zoom for free tiles)
 |      - "Mapbox Control Room" (Limited levels of zoom for free tiles)
 |      - "Stamen" (Terrain, Toner, and Watercolor)
 |      - "Cloudmade" (Must pass API key)
 |      - "Mapbox" (Must pass API key)
 |      - "CartoDB" (positron and dark_matter)
 |  
 |  You can pass a custom tileset to Folium by passing a Leaflet-style
 |  URL to the tiles parameter: ``http://{s}.yourtiles.com/{z}/{x}/{y}.png``
 |  
 |  Parameters
 |  ----------
 |  location: tuple or list, default None
 |      Latitude and Longitude of Map (Northing, Easting).
 |  width: pixel int or perc

In [8]:
# create map
# m = folium.Map(location=location, zoom_start=zoom_start, )
m = folium.Map(location=location, tiles="stamentoner", zoom_start=zoom_start)
hm = HeatMap(data.tolist(), min_opacity=min_opacity, blur=blur, radius=radius)

hm.add_to(m)
# folium.GeoJson(district23).add_to(hmap)
# hmap.add_child(hm_wide)

m.save(os.path.join('results', 'Heatmap_1.html'))
hm.save(os.path.join('results', 'geo_heatmap.html'))


NameError: name 'HeatMap' is not defined

# Saving png's of Folium Maps

https://github.com/python-visualization/folium/issues/35

These aren't tested, just pasted here as a starting point should someone need them.

### Using cutycapt

In [39]:
# save to png file
outdir = "results" # this directory has to exist..
#hmap.save("tmp.html")  # heatmap saved above
url = "file://{}/results/geo_heatmap.html".format(os.getcwd())
outfn = os.path.join(outdir,"geo_heatmap.png")
subprocess.check_call(["cutycapt","--url={}".format(url), "--out={}".format(outfn)])

0

### Using selenium

In [35]:
import os
import time
from selenium import webdriver

fn = os.path.join('results', 'geo_heatmap.html')
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
#hmap.save("tmp.html")  # heatmap saved above

browser = webdriver.Firefox()
browser.get(tmpurl)
time.sleep(5)  #Give the map tiles some time to load
browser.save_screenshot('geo_heatmap.png')
browser.quit()

WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 
